# Binaryzacja

### Zadanie domowe - binaryzacja adaptacyjna w oknach z interpolacją.

Pokazana w ramach podstawowej części ćwiczenia binaryzacja adaptacyjna działa dobrze, ale jest dość złożona obliczeniowo (choć oczywiście należy mieć świadomość, że implementację można zoptymalizować i wyeliminować pewne powtarzające się obliczenia).
Zbliżone rozwiązanie można również realizować w nieco innym wariancie - w oknach.
Ogólna idea jest następująca: wejściowy obraz dzielimy na nienachodzące (rozłączne) okna - wygodnie jest założyć, że są one kwadratowe i o rozmiarze będącym potęgą liczby 2.
W każdym z okien obliczamy próg - niech to będzie średnia i stosujemy do binaryzacji lokalnej.
Jak nietrudno się domyślić efekt nie będzie dobry, gdyż na granicach okien wystąpią artefakty.
Aby je wyeliminować należy zastosować interpolację, co zostanie szczegółowo opisane poniżej.
Warto zaznaczyć, że podobny mechanizm interpolacji stosowany jest w poznanym wcześniej algorytmie CLAHE.
Zainteresowane osoby odsyłam do artykułu na [Wikipedii](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization) oraz do artykułu o metodzie CLAHE - Zuiderveld, Karel. “Contrast Limited Adaptive Histograph Equalization.” Graphic Gems IV. San Diego: Academic Press Professional, 1994. 474–485.



Na początek zaimplementujemy wariant metody bez interpolacji:
1. Wczytaj obraz _rice.png_.
2. W dwóch pętlach `for`, dla okien o ustalonym wymiarze $W$ (potęga 2), oblicz średnią:
- pętle powinny mieć krok $W$,
- wynik (tj. średnie) należy zapisać w pomocniczej tablicy,
- przydatny operator to `//` - dzielenie całkowitoliczbowe (*floor division*).

3. W kolejnych dwóch pętlach `for` (tym razem o kroku 1) przeprowadź binaryzację z wyznaczonymi progami.
   Tu oczywiście należy się sprytnie odwołać do wyników z tablicy pomocniczej.
   Wyświetl wyniki - czy jest on poprawny?
   Podpowiedź - błędy lepiej widać dla mniejszego rozmiaru okna (np. 16 x 16).

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/'

fileName = "rice.png"
if not os.path.exists(fileName):
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
def show_image(image):
    plt.imshow(image, 'gray')
    plt.axis('off')

In [ ]:
rice = cv2.imread("rice.png", cv2.IMREAD_GRAYSCALE)
show_image(rice)

In [ ]:
def window_means(image, w):
    x, y = image.shape
    means = np.zeros((x // w, y // w))
    for i in range(0, x, w):
        for j in range(0, y, w):
            means[i // w, j // w] = np.mean(image[i:i+w, j:j+w])

    return means


def adaptive(image_name, w):
    image = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)
    x, y = image.shape
    means = window_means(image, w)
    binary = np.zeros(image.shape)
    for i in range(x):
        for j in range(y):
            binary[i, j] = 0 if image[i, j] <= means[i // w, j // w] else 255
    return binary


show_image(adaptive("rice.png", 16))

In [ ]:
show_image(adaptive("rice.png", 32))

In [ ]:
show_image(adaptive("rice.png", 64))

4. Rozwiązaniem problemu artefaktów na obrazie jest zastosowanie interpolacji.
   Próg binaryzacji dla danego piksela wyliczany jest na podstawie progów z sąsiednich okien.
   ![Ilustracja koncepcji interpolacji](https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/clahe_tile_interpolation.png)

   Koncepcja została przedstawiona na powyższym rysunku.
   Możliwe są 3 przypadki:
   - piksel leży w rogach obrazu (kolor czerwony) - wtedy za próg przyjmuje się wartość średniej obliczonej dla danego okna,
   - piksel leży na krawędzi obrazu (kolor zielony) - wtedy za próg przyjmuje się wartość obliczoną na podstawie średnich z dwóch sąsiednich okien,
   - piksel leży w środku (kolor fioletowy) - wtedy próg jest obliczany na podstawie 4 sąsiednich okien.

   Uwaga. Proszę zwrócić uwagę, że sprawa jest dość złożona.
   Obraz dzielimy na okna (dla nich liczymy średnią) i następnie każde z okien "wirtualnie" na cztery sub-okna (linie przerywane).
   To ułatwia znalezienie środków okien (czarne kwadraty), które są wykorzystywane w interpolacji.

5. Implementujemy interpolację.
   Potrzebujemy do tego znać progi (jeden, dwa lub cztery), ale dla przejrzystości obliczeń lepiej zawsze przyjąć cztery oraz odległości od rozważnego piksela do środka sąsiednich okien (też w ogólnym przypadku 4):
   - całość sprowadza się do określania pozycji piksela,
   - na początek rozważmy przypadek czterech narożników (kolor czerwony na rysunku) - trzeba napisać `if`, który je wyznaczy,
   - warto sprawdzić, czy nie popełniliśmy błędu i np. tymczasowo do obrazu wynikowego w tym miejscu przypisać wartość 255. Efekt powinien być taki, że widoczne będą tylko narożniki.
   - drugi przypadek do brzegi (kolor zielony) - postępujemy podobnie jak przy narożnikach, przy czym osobno wydzielamy brzegi pionowe i poziome. Tu też warto sobie obrazek "pokolorować".
   - na koniec wyznaczamy piksele w środku.
   - analizując poprawność proszę zwrócić uwagę na to, żeby nie było przerw pomiędzy obszarami.
   - mając podział możemy dla każdego z obszarów wyliczyć cztery progi ($t11, t12, t21, t22$):
        - dla narożników wartość ta będzie identyczna i wynosi po prostu `t11 =t[jT][iT]`, gdzie `iT=i//W` oraz `jT=j//W`.
          Uwaga. Proszę używać indeksów tymczasowych $jT,iT$, gdyż będą potrzebne w dalszych obliczeniach.
        - dla brzegów pionowych występują dwie wartości: okno bieżące i sąsiednie.
          Wyznaczenie współrzędnej poziomej jest proste (jak dla narożników).
          Nad współrzędną pionową trzeba się chwilę zastanowić - aby nie rozważać wielu przypadków można od bieżącej współrzędnej odjąć połowę rozmiaru okna i dopiero później wykonać dzielenie przez rozmiar okna.
          W ten sposób otrzymujemy indeks okna o mniejszej współrzędnej.
          Indeks drugiego uzyskamy dodając 1.
          Proszę się zastanowić dlaczego to działa - najlepiej to sobie rozrysować.
        - dla brzegów poziomych należy postąpić analogicznie,
        - obliczenia dla obszaru wewnątrz powinny być już oczywiste.
   - kolejny krok to wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami.
     Przykładowo dla osi X wygląda to następująco: `dX1 = i - W/2 - iT*W` oraz `dX2 = (iT+1)*W - i-W/2`.
     Dla osi Y analogicznie.
     Ponownie proszę się zastanowić dlaczego to jest poprawne - najlepiej to sobie narysować.
   - ostatni krok to interpolacja dwuliniowa.
     Wykonamy ją w trzech krokach:
     - interpolacja w osi X dla dwóch górnych okien - sprowadza się ona do średniej ważonej pomiędzy wartościami $t11$ i $t12$, przy czym wagi to odpowiednio $dX2/W$ i $dX1/W$.
       Ponownie na podstawie rysunku proszę to przemyśleć.
     - interpolacja w osi X dla dolnych okien jest analogiczna,
     - interpolacja w osi Y również jest analogiczna, z tym, że wejściem są dwa wyniki interpolacji w poziomie.

6. "Kropka nad i" to oczywiście binaryzacja z wyznaczonym poprzez interpolację progiem - proszę dobrać rozmiar okna.
7. Na koniec proszę porównać na wspólnym rysunku poznane metody binaryzacji:
- Otsu,
- lokalna na podstawie średniej,
- lokalna Sauvoli,
- lokalna w oknach bez interpolacji,
- lokalna w oknach z interpolacją.

Proszę pod porównaniem, w osobnej sekcji *markdown*, krótko skomentować uzyskane wyniki.

In [ ]:
def interpolation(image_name, w):

    def corner(x1, y1):
        mean = means[x1 // w, y1 // w]
        binary[x1, y1] = 0 if image[x1, y1] <= mean else 255

    def dist_center(cord, rect_num):
        return abs(cord - rect_num*w - w2)

    def find_window(cord):
        cord = (cord - w2) // w
        return cord, cord + 1

    image = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)
    x, y = image.shape
    means = window_means(image, w)
    binary = np.zeros(image.shape)
    w2 = w // 2

    for i in range(w2):
        for j in range(w2):
            corner(i, j)
            corner(x - i - 1, y - j - 1)
            corner(i, y - j - 1)
            corner(x - i - 1, j)

    for right in [True, False]:
        for i in range(w2, x - w2):
            for j in range(w2):
                if right:
                    j = y - j - 1
                m1, m2 = find_window(i)
                t = (dist_center(i, m2) / w)*means[m1, 0] + (dist_center(i, m1) / w)*means[m2, 0]
                binary[i, j] = 0 if image[i, j] <= t else 255

    for up in [True, False]:
        for j in range(w2, y - w2):
            for i in range(w2):
                if up:
                    i = x - i - 1
                m1, m2 = find_window(j)
                t = (dist_center(j, m2) / w)*means[0, m1] + (dist_center(j, m1) / w)*means[0, m2]
                binary[i, j] = 0 if image[i, j] <= t else 255

    for i in range(w2, x - w2):
        for j in range(w2, y - w2):
            m1, m2 = find_window(i)
            m3, m4 = find_window(j)
            dist1, dist2 = dist_center(j, m3), dist_center(j, m4)
            t1 = (dist2 / w)*means[m1, m3] + (dist1 / w)*means[m1, m4]
            t2 = (dist2 / w)*means[m2, m3] + (dist1 / w)*means[m2, m4]

            t = (dist_center(i, m2) / w)*t1 + (dist_center(i, m1) / w)*t2
            binary[i, j] = 0 if image[i, j] <= t else 255

    return binary

In [ ]:
show_image(interpolation("rice.png", 16))

In [ ]:
show_image(interpolation("rice.png", 32))

In [ ]:
show_image(interpolation("rice.png", 64))

In [ ]:
def otsu(image_name):
    image = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist = np.squeeze(hist) / np.sum(hist)

    cumulative = np.cumsum(hist)
    means = np.cumsum(hist * np.arange(256))
    mean_all = means[-1]

    variance = np.divide(np.square(mean_all * cumulative - means), (cumulative * (1 - cumulative)), where=((0 < cumulative) & (cumulative < 1)))
    opt_k = np.argmax(variance)
    thresh_otsu, _ = cv2.threshold(image, 0, 255, cv2.THRESH_OTSU)
    image[image <= opt_k] = 0
    image[image > opt_k] = 255
    return image

def local_mean(image_name, window_size):
    image = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)
    X, Y = image.shape
    W = window_size // 2
    binary = np.zeros(image.shape)
    for i in range(W, Y - W):
        for j in range(W, X - W):
            window = image[i-W:i+W, j-W:j+W]
            binary[i, j] = 0 if image[i, j] <= np.mean(window) else 255
    return binary

def local_sauvola(image_name, window_size, k=0.15, R=128, sign=-1):
    image = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)
    X, Y = image.shape
    W = window_size // 2
    binary = np.zeros(image.shape)
    for i in range(W, Y - W):
        for j in range(W, X - W):
            window = image[i-W:i+W, j-W:j+W]
            T = np.mean(window) * (1 + sign*k*(np.std(window) / R - 1))
            binary[i, j] = 0 if image[i, j] <= T else 255
    return binary

In [ ]:
def compare_thresholding(image_name):
    im1 = adaptive(image_name, 32)
    im2 = interpolation(image_name, 32)
    im3 = otsu(image_name)
    im4 = local_mean(image_name, 25)
    im5 = local_sauvola(image_name, 16)

    fig, ax = plt.subplots(3, 2)
    fig.set_size_inches(12, 15)

    ax[0, 0].imshow(im1, 'gray')
    ax[0, 0].axis('off')
    ax[0, 0].set_title("Local without interpolation")

    ax[0, 1].imshow(im2, 'gray')
    ax[0, 1].axis('off')
    ax[0, 1].set_title("Interpolation")

    ax[1, 0].imshow(im3, 'gray')
    ax[1, 0].axis('off')
    ax[1, 0].set_title("Otsu")

    ax[1, 1].imshow(im4, 'gray')
    ax[1, 1].axis('off')
    ax[1, 1].set_title("Local with mean")

    ax[2, 0].imshow(im5, 'gray')
    ax[2, 0].axis('off')
    ax[2, 0].set_title("Sauvola")


    ax[2, 1].axis('off')


In [ ]:
compare_thresholding("rice.png")

Widać, że najsłabiej radzą sobie algorytmy które bazują tylko na liczeniu średniej (dla każdego punktu lub okna). Obrazy po tym przekształceniu zawierają najwięcej szumu.
Algorytm Otsu praktycznie całkowicie wyeliminował szum, lecz kosztem utraty informacji o kilku ziarnkach ryżu.
ALgorytm Sauvola i Interpolacja wyeliminowały większość szumu zostawiając przy tym jak najwięcej informacji o interesujących nas obiektach.